# Readme

This is a baseline model, which only uses SBERT-based semantic search without TFIDF or any preprocessing

# Preparation

In [46]:
from sentence_transformers import SentenceTransformer
import scipy
import os
import pandas as pd
import numpy as np
from stemming.porter2 import stem
import re
import csv

# preprocessing method
def string_tokenise(string):  # return list
    result = re.findall(r"\w+", string)
    return result


def case_fold(list1):  # return list
    result = [word.lower() for word in list1]
    #     string = ' '.join([str(elem) for elem in list1])
    #     result = string.lower().split() #lower() is the same as casefold()
    return result


def stopping(list1):  # return list
    stopfile = open("englishST.txt", 'r')
    stopwords = stopfile.read().split()
    result = [items for items in list1 if items not in stopwords]
    return result


def normalise(list1):  # return list
    result = []
    for item in list1:
        result.append(stem(item))
    return result

# LOAD BERT SENTENCE MODEL
# Load the BERT model. Various models trained on Natural Language Inference (NLI) https://github.com/UKPLab/sentence-transformers/blob/master/docs/pretrained-models/nli-models.md and 
# Semantic Textual Similarity are available https://github.com/UKPLab/sentence-transformers/blob/master/docs/pretrained-models/sts-models.md

model = SentenceTransformer('bert-base-nli-mean-tokens')

# try different encoding and seperate identifiers
def read_csv(filepath):
     if os.path.splitext(filepath)[1] != '.csv':
          return  # or whatever
     seps = [',', ';', '\t']                    # ',' is default
     encodings = [None, 'utf-8', 'ISO-8859-1', 'utf-16','ascii']  # None is default
     for sep in seps:
         for encoding in encodings:
              try:
                  return pd.read_csv(filepath, encoding=encoding, sep=sep)
              except Exception:  # should really be more specific 
                  pass
     raise ValueError("{!r} is has no encoding in {} or seperator in {}"
                      .format(filepath, encodings, seps))

# Set up (and preprocess) and Embedding

In [18]:
# parse the verified claims
vclaims_directory = 'v3.0/verified_claims.docs.tsv'
vclaims_fields = ['vclaim_id', 'vclaim', 'title']
df_vc = pd.read_csv(vclaims_directory, usecols = vclaims_fields, sep = '\t')
vclaims_list = df_vc.vclaim.tolist().copy()
vclaims_list

# prepare queries and put them in a list
tweets_directory = 'v3.0/train/tweets.queries.tsv'
tweets_fields = ['tweet_id', 'tweet_content']
df_t = pd.read_csv(tweets_directory, usecols = tweets_fields, sep = '\t')
tweets_list = df_t.tweet_content.tolist().copy()
tweets_list



['Trump needs to immediately divest from his businesses and comply with the emoluments clause. Iran could threaten Trump hotels *worldwide* and he could provoke war over the loss of revenue from skittish guests.  His business interests should not be driving military decisions. — Ilhan Omar (@IlhanMN) January 6, 2020',
 'A number of fraudulent text messages informing individuals they have been selected for a military draft have circulated throughout the country this week.',
 'Fact check: The U.S. Army is NOT contacting anyone regarding the draft. If you are receiving texts, phone calls or direct messages about a military draft, they are not official communications from the U.S. Army pic.twitter.com/3S32De8ekP — U.S. Army CGSC (@USACGSC) January 8, 2020',
 'The US drone attack on #Soleimani caught on camera.#IranUsapic.twitter.com/TvRkHvlgby — Olaudah Equiano® (@RealOlaudah) January 6, 2020',
 '1. To the dim witted reporters like @dmedin11:  no one said this wasn’t photoshopped.  No one 

In [19]:
# Use SBERT to do sentence embedding
# Each vclaim is encoded as a 1-D vector with 78 columns
vclaim_embeddings = model.encode(vclaims_list)

# print('Sample BERT embedding vector - length', len(vclaim_embeddings[0]))

np.save('06_embeddings.npy', vclaim_embeddings)
print('Sample BERT embedding vector - note includes negative values', vclaim_embeddings[0])

Sample BERT embedding vector - note includes negative values [-0.06839999  0.11646768 -0.37378955  0.27000183  0.37912527 -1.2374749
  0.5467199  -0.64416355  0.01363655 -0.06171849 -0.23101607 -0.4370634
 -0.11530093  0.78729784 -0.53884095  0.30604655 -0.654617   -0.11056169
 -0.00810371 -0.20159982  0.39454025 -0.6258818  -0.50732946 -0.26715767
  0.86845887  0.3733293  -0.40977922 -0.76450676 -1.4386698   0.29790646
 -0.49806467  0.2177606  -0.03685323 -0.16548534 -0.138993    0.7363141
 -1.177447   -0.17506939 -0.21862906 -0.26186585 -0.07425258 -0.25737232
 -0.6412997  -0.24607065 -0.792142   -0.5476829   0.7522482   0.46411932
  0.578255   -0.30182254  0.29880628 -0.02058476 -0.3748311  -0.7059333
  0.07826146 -0.19672419  0.09062672 -1.7489096  -0.27156785 -0.5553281
 -0.6658741   0.36642843  0.52788115  0.60650444  0.26296413 -0.00902202
  0.4893662   0.8471768  -1.2924407  -1.1877214   0.49134386 -0.5718257
 -0.7279378  -0.45531315  0.3747597  -0.25155854 -0.16524307 -0.14369

# PERFORM SEMANTIC SEARCH AND GENERATE RESULTS

In [52]:
# prepare embeddings
copy = np.load('01_embeddings.npy')
vclaim_embeddings = copy.copy() # duplicate embedding data
tweet_embeddings = model.encode(tweets_list)

# Find the closest 3 sentences of the corpus for each tweet sentence based on cosine similarity
number_top_matches = 20 #@param {type: "number"}

print("Semantic Search Results")

for tweet, tweet_embedding in zip(tweets_list, tweet_embeddings):
    return_data = [] # a dict that contains the info that needed to be reported
                    # in eval. the structure is {'attr1': value1, 'attr2': value2}
    distances = scipy.spatial.distance.cdist([tweet_embedding], vclaim_embeddings, "cosine")[0]

    results = zip(range(len(distances)), distances)
    results = sorted(results, key=lambda x: x[1])

    print("\n\n======================\n\n")
    print("tweet:", tweets_list[tweets_list.index(tweet)]) # for best reading, output orginial text reather than tokens
    print("\nTop 20 most similar vclaim in corpus:")

    rank = 0 # initialize param rank in return_data
    for idx, distance in results[0:number_top_matches]:
        print(vclaims_list[idx].strip(), "(Cosine Score: %.4f)" % (1-distance))
        score = "%.4f" % (1-distance) # an parameter that need to be reported in eval
        
        # write the results to the file
        tweet_id = df_t.loc[df_t['tweet_content'] == tweet,'tweet_id'].item()
        vclaim_id = df_vc.loc[df_vc['vclaim'] == vclaims_list[idx],'vclaim_id'].item()
        rank = rank + 1
        score = score
        tag = 'DC'

        return_data = {'tweet_id':tweet_id, 'Q0':'Q0', 'vclaim_id':vclaim_id, 'rank': '1',
                      'score': score, 'tag': tag}
        # NB: no matter what the real rank is, it should be 1 if you want it to be counted
    
        # write data to the results file
        # results_fields = ['tweet_id','Q0','vclaim_id','rank','score','tag']
        # NO HEADER NEEDED!!
        results_filename = '01_results.tsv'
        with open(results_filename, 'a') as csvfile:
            writer = csv.DictWriter(csvfile, fieldnames=results_fields, delimiter='\t')
            writer.writerow(return_data)
    

Semantic Search Results




tweet: Trump needs to immediately divest from his businesses and comply with the emoluments clause. Iran could threaten Trump hotels *worldwide* and he could provoke war over the loss of revenue from skittish guests.  His business interests should not be driving military decisions. — Ilhan Omar (@IlhanMN) January 6, 2020

Top 20 most similar vclaim in corpus:
Donald Trump said that America's economy crashing would lead to riots demanding we make the country great again. (Cosine Score: 0.8107)
Major General Paul Eaton said President Trump's decertification of an existing nuclear deal with Iran deal 'dishonors America.' (Cosine Score: 0.8028)
Major General Paul Eaton said President Trump's decertification of an existing nuclear deal with Iran deal "dishonors America." (Cosine Score: 0.8026)
London's new mayor Sadiq Khan threatened Donald Trump with terror attacks if the candidate's proposed anti-Islam proposals were implemented. (Cosine Score: 0.8022)
Celebrit





tweet: Picture of engagement rings removed from the fingers of World War- II victims. Imagine how many love stories were buried!Wars are never easy!Nd we r making fun of it.. #shameful act..#WWIIl#WW3pic.twitter.com/9CgOACecb0 — ChaudhRy Saab (@tera_Lover_) January 5, 2020

Top 20 most similar vclaim in corpus:
President Bush misspoke at a right-to-life rally and repeatedly said ‘feces’ instead of ‘fetus.’ (Cosine Score: 0.7680)
A leaked e-mail revealed Clinton aide Cheryl Mills calling the men who died in Benghazi "idiot soldiers" and saying she was glad they were tortured. (Cosine Score: 0.7632)
Pat Robertson said that Hurricane Katrina was caused by God’s anger over the selection of lesbian comedienne Ellen Degeneres to host the upcoming Emmy Awards. (Cosine Score: 0.7625)
Donald Trump divorced Ivana Trump because her breast implants were imperfect, and he leaked that information to New York tabloids while they were married. (Cosine Score: 0.7572)
Bride’s mix-up results in an em





tweet: Once more for the folks who missed the tweet the first time I posted it. Minecraft is NOT stopping, Mojang is NOT closing. #Minecrafthttps://t.co/V4Lzv97qOc — Helen Z 🔜 #PAXSouth (@HelenAngel) January 2, 2020

Top 20 most similar vclaim in corpus:
In a 2 January 2018 Twitter thread, Chelsea Clinton wished the Church of Satan a Happy New Year. (Cosine Score: 0.7076)
Yahoo! will begin charging for its previously-free POP e-mail service in April 2002. (Cosine Score: 0.6954)
You can avoid a recent AOL price increase for unlimited access subscriptions by calling and asking to be locked in at the old rate for one year. (Cosine Score: 0.6702)
Hillary Clinton overlooked Benghazi victims when she said that 'we didn't lose a single person in Libya' during a campaign event on MSNBC. (Cosine Score: 0.6503)
Dairy Queen is hosting 'Free Cone Day' on 20 March 2017. (Cosine Score: 0.6445)
Bernie Sanders and Hillary Clinton both seek an abortion 'cutoff date' of 36 weeks, and both claim that





tweet: This photo (A Menorah Defies Nazi Rule, 1933) was taken as the Nazis were rising to power, a symbol of the belief that the Jews could and would survive anything. The risk the person taking it must have faced is almost impossible to imagine. Happy #Hanukkahpic.twitter.com/IfkdcwGENr — Abel Eve Adam (@abel_eve_adam) December 23, 2019

Top 20 most similar vclaim in corpus:
Adolf Hitler said that the only way that the Nazi party could have been stopped was if it were destroyed in its infancy. (Cosine Score: 0.7285)
Sarah Silverman said 'I hope the Jews did kill Jesus, I'd do it again in a second!' (Cosine Score: 0.7099)
Democrats at Elizabethtown College mirrored the Nazi policy of forcing Jewish people to wear the Star of David in a February 2017 campaign that had students wear 'white privilege' pins. (Cosine Score: 0.7093)
During World War II, the Germans passed up a chance to kill Franklin D. Roosevelt because they mistranslated the name of the city Casablanca as “White House





tweet: While we removed the Kingdom of Wakanda from our list of US free trade partners, our relationship will always be strong #WakandaForeverpic.twitter.com/wiRSCIdfGh — Dept. of Agriculture (@USDA) December 19, 2019

Top 20 most similar vclaim in corpus:
Dairy Queen is hosting 'Free Cone Day' on 20 March 2017. (Cosine Score: 0.6888)
Donald Trump promised on Twitter that his first act as president would be to pardon Dylann Roof. (Cosine Score: 0.6834)
Between January 2017 and March 2018, Diahann Carroll publicly praised and defended President Donald Trump and pleaded with Democrats to end their criticism of him. (Cosine Score: 0.6689)
During remarks in February 2019, Trump touted the 'abolition of civil rights' as one of the greatest accomplishments in the history of the United States. (Cosine Score: 0.6625)
An autumn 2018 outbreak of E. coli linked to romaine lettuce was caused by a decision by President Trump to reverse existing rules on agricultural water inspection. (Cosine Sc

...





tweet: Since @Redskins name is offensive, we should change the @CrackerBarrel name next. #RedskinsNationpic.twitter.com/v9rWHs61YD — David (@dpartenio) July 10, 2015

Top 20 most similar vclaim in corpus:
Bruce Springsteen called Donald Trump an 'asshole' on Twitter, signing off with the pro-Clinton hashtag #ImWithHer. (Cosine Score: 0.7518)
CNN reported that Twitter is deleting Donald Trump's account over its use of 'racist rhetoric.' (Cosine Score: 0.7472)
Rachel Maddow tweeted that Donald Trump needs to be 'taken out' before he destroys America. (Cosine Score: 0.7393)
Barack Obama poked fun at President Trump on Twitter for saying that grocery shoppers required ID, (Cosine Score: 0.7313)
A new 'finger chop' challenge is spreading on social media in protest of President Trump. (Cosine Score: 0.7202)
Donald Trump sent a tweet with the word "unprecedented" incorrectly spelled "unpresidented." (Cosine Score: 0.7191)
Official Department of Interior Twitter accounts were temporarily d





tweet: @cenkuygur hillary campaign wont let me cancel recurring donations! 6+ emails and no response HELP!! — CW (@genesntees) January 7, 2016

Top 20 most similar vclaim in corpus:
In 2016, Senate Majority Leader Mitch McConnell said: 'One of my proudest moments was when I told Obama, 'You will not fill this Supreme Court vacancy,'' and in 2017, he said, 'Apparently there's yet a new standard now, which is not to confirm a Supreme Court nominee at all. I think that's something the American people simply will not tolerate.' (Cosine Score: 0.7959)
The National Park Service specifically blocked the Million Women March as well as all other demonstrations through March 2017. (Cosine Score: 0.7731)
Vice President Joe Biden was refused service at the Crumb and Get It cookie shop in 2012. (Cosine Score: 0.7664)
Democrats Bernie Sanders, Keith Ellison, Debbie Wasserman Schultz, and Nancy Pelosi refused to stand and clap for Navy SEAL widow Carryn Owens during a presidential speech. (Cosine

In December 2018, Trump 2020 campaign advisor Katrina Pierson demanded that Sen. John McCain "answer questions" -- four months after his death. (Cosine Score: 0.7504)
Emma Breier, the daughter of an American diplomat, was used as a 'crisis actor' for U.S. President Donald Trump's hospital visit after the Dayton mass shooting in August 2019. (Cosine Score: 0.7504)
Sen. David Perdue advised attendees to offer up an ominous prayer aimed at President Obama during the 2016 Faith & Freedom Coalition’s Road to Majority conference. (Cosine Score: 0.7447)
Frito-Lay issued a recall of two potato chip products in April 2017 over Salmonella-related fears, a report framed as new in July and August 2017. (Cosine Score: 0.7433)
In May 2019, the city of Statesville, N.C., filed for a court injunction to force Camping World to remove an American flag from their local branch. (Cosine Score: 0.7369)
A sign in Calvert County, Maryland was erected in May 2018, warning 'liberals' to arm themselves in the ev





tweet: Giants stars dare NFL to fine them for cleats in tribute of 9/11 https://t.co/HMxA64RpQL via @nypostsports — michael (@michael75058302) September 12, 2016

Top 20 most similar vclaim in corpus:
The Philadelphia Eagles responded to a tweet from President Trump congratulating the team on winning the Super Bowl by telling the Commander-in-Chief to go f*ck himself. (Cosine Score: 0.8031)
An NFL owner told his players they were 'actors' playing the role of 'patriotic super heroes' and therefore had to stand during the playing of the U.S. national anthem. (Cosine Score: 0.7657)
A fan’s tweets verified the results of the 2014-15 NFL season and Super Bowl were rigged. (Cosine Score: 0.7567)
Nike captured an amazing photo of the coin toss before the 2015 College Football Playoff championship game. (Cosine Score: 0.7440)
Image shows an advertisement from the Virginia Beach SPCA thanking quarterback Michael Vick for his assistance in rescuing pit bulls. (Cosine Score: 0.7429)
Tom Brady





tweet: #BreakingNews: We’re launching an exciting new savoury #condom range – Eggplant flavour! ? #CondomEmojipic.twitter.com/idA07EaiXC — Durex Global (@durex) September 5, 2016

Top 20 most similar vclaim in corpus:
The "Tide pod challenge" is a real viral phenomenon whereby people bite into Tide brand laundry detergent pods. (Cosine Score: 0.7139)
The 'Tide pod challenge' is a real viral phenomenon whereby people bite into Tide brand laundry detergent pods. (Cosine Score: 0.7088)
A video surfaced in October 2016 showing President Obama demanding 'ribs and pussy.' (Cosine Score: 0.7086)
Target issued a recall for a line of Easter egg toys. (Cosine Score: 0.7004)
Krispy Kreme is giving free doughnuts to anyone who talks or dresses like a pirate on 19 September 2016. (Cosine Score: 0.7001)
The Boy Scouts implemented a new policy requiring condoms to be available at its quadrennial World Scout Jamboree, shortly after announcing that girls would be able to join the organization. (Cos

In [44]:
df_t.loc[df_t['tweet_content'] == 'NYPD just came to my house bc Rutgers Police told them i’m a threat based on political statements i’ve made on campus and on twitter. — Kevin Allred (@KevinAllred) November 16, 2016','tweet_id'].item()

896